<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/Parseo_de_Discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs

Mounted at /content/drive
 80% 42.0M/52.3M [00:00<00:00, 65.3MB/s]
100% 52.3M/52.3M [00:00<00:00, 73.4MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scrap PDFs.csv')

In [47]:
import pandas as pd
pd.options.mode.chained_assignment = None
def generar(l):
  lineas = l[2].strip().split("\n")

  coincidencias_nombre_cargo = []
  coincidencias_discurso = []
  discurso_actual = ""

  for linea in lineas:
      if linea[0].isupper() and (":" in linea or "(" in linea):
          coincidencias_nombre_cargo.append(linea.split(":")[0].strip())
          coincidencias_discurso.append(discurso_actual)
          discurso_actual = linea.strip()
      else:
          discurso_actual += " " + linea.strip()

  # Agregar la última coincidencia de discurso
  coincidencias_discurso.append(discurso_actual)

  # Verificar si hay la misma cantidad de coincidencias para nombre/cargo y discursos
  if len(coincidencias_nombre_cargo) == len(coincidencias_discurso[1:]):
      # Crear el DataFrame de pandas
      data = {
          "Nombre/Cargo": coincidencias_nombre_cargo,
          "Discurso": coincidencias_discurso[1:]
      }
      df = pd.DataFrame(data)
      df["Discurso"] = df["Discurso"].str.strip()

  else:
      print("Error: no coincide la cantidad de coincidencias para nombre/cargo y discursos.")

  df = df[df['Nombre/Cargo'].str.contains(r'[A-Z]{2,}')]
  df['Discurso'] = df.apply(lambda row: re.sub(re.escape(row['Nombre/Cargo']) + r':\s*', '', row['Discurso']), axis=1)
  df['Cargo'] = df['Nombre/Cargo'].apply(lambda x: ' '.join(re.findall(r'\b[A-Z][A-ZÁÉÍÓÚÜÑ-]+\b', x)))
  df['Nombre'] = df['Nombre/Cargo'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0] if re.findall(r'\((.*?)\)', x) else '')
  df['Link']=l[1]
  df['Legislatura']=l[1][27:29]
  return df

In [48]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo', 'Nombre'])

In [49]:
import re
for row in base.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [50]:
base2=base2.reset_index(drop=True)

In [51]:
base2

,Nombre/Cargo,Discurso,Cargo,Nombre,Link,Legislatura
0,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,Se abre la sesión.,PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10
1,EL señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,En cumplimiento de lo establecido en el aparta...,EL PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10
2,El señor LETRADO MAYOR,"Con su permiso, señor presidente. Real Decreto...",LETRADO MAYOR,,https://www.senado.es/legis10/publicaciones/pd...,10
3,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,"Muchas gracias, señor letrado mayor. FORMACIÓN...",PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10
4,El señor PRESIDENTE DE LA JUNTA PREPARATORIA (...,"A continuación, de acuerdo con el artículo 3 d...",PRESIDENTE DE LA JUNTA PREPARATORIA,Alcalde Sánchez,https://www.senado.es/legis10/publicaciones/pd...,10
...,...,...,...,...,...,...
116936,La señora VICEPRESIDENTA (Narbona Ruiz),Se vota en sus propios términos.,VICEPRESIDENTA,Narbona Ruiz,https://www.senado.es/legis14/publicaciones/pd...,14
116937,La señora VICEPRESIDENTA (Narbona Ruiz),Queda rechazada.Núm. 135 Pág. 210 17 de mayo d...,VICEPRESIDENTA,Narbona Ruiz,https://www.senado.es/legis14/publicaciones/pd...,14
116938,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,FINANCIACIÓN AUTONÓMICA EN TÉRMINOS DE UNA MAY...,Votación,https://www.senado.es/legis14/publicaciones/pd...,14
116939,La señora VICEPRESIDENTA (Narbona Ruiz),"Se vota en los términos de la transaccional, c...",VICEPRESIDENTA,Narbona Ruiz,https://www.senado.es/legis14/publicaciones/pd...,14
